In [1]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np
import sys

import seaborn as sns
from matplotlib import cm
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
%matplotlib inline

from spacy.tokens import Token
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk.corpus import stopwords

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import silhouette_score
from sklearn import metrics

from sklearn.model_selection import ShuffleSplit

import gensim
from gensim import corpora, models
from gensim.models.phrases import Phrases, Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec
import multiprocessing

from sklearn.model_selection import cross_val_score , GridSearchCV,train_test_split

from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

import enchant
pd.set_option('display.max_rows', 500)

In [45]:
dict_check = enchant.Dict("en_US")

#### Importing the file ####
Path="src/"

Filename='projects_Preprocessed.csv'
df=pd.read_csv(Path+Filename)

DBFilename='Report_WEPCADBScanFindingsKMeansV3_Exp1.csv'
DB_df=pd.read_csv(Path+DBFilename)

ISOForestFilename='Report_WEPCAISOForestFindingsKMeansV3.csv'
ISO_df=pd.read_csv(Path+ISOForestFilename)

manualtag=pd.read_csv(Path+'SamplesManualTagger.csv')

df['SubUrl']='https://gepris.dfg.de'+df['SubUrl']

## if it is interdiscipilinary then -1 otherwise 0 (Normal data)
manualtag['interdiscipilinary']=-1
manualtag.loc[manualtag.apply(lambda x: (x['Category']==x['Category_1']) & (pd.isnull(x['Category_2'])) , axis=1),'interdiscipilinary']=1

In [5]:
DB_df.interdiscipilinary.value_counts()

False    49410
True      1623
Name: interdiscipilinary, dtype: int64

In [6]:
ISO_df.interdiscipilinary.value_counts()

False    49529
True      1504
Name: interdiscipilinary, dtype: int64

In [48]:
Merged_df=ISO_df[['ProjectURL','Category','Category_1_Values', 'Category_2_Values','Category_1', 'Category_2','ISOForestCluster','interdiscipilinary'] ].merge(DB_df[['ProjectURL','Category_1_Values', 'Category_2_Values', 'Category_1', 'Category_2','DBScanCluster','interdiscipilinary' ]], how='inner', left_on='ProjectURL', right_on='ProjectURL',suffixes= ('_ISO','_DB'))
Merged_df=Merged_df.merge(df[['SubUrl','Translates']],how='left',left_on='ProjectURL',right_on='SubUrl')
Feature=['Translates','Category','Category_1_DB','Category_1_ISO','Category_2_DB','Category_2_ISO']

In [49]:
Merged_df.columns

Index(['ProjectURL', 'Category', 'Category_1_Values_ISO',
       'Category_2_Values_ISO', 'Category_1_ISO', 'Category_2_ISO',
       'ISOForestCluster', 'interdiscipilinary_ISO', 'Category_1_Values_DB',
       'Category_2_Values_DB', 'Category_1_DB', 'Category_2_DB',
       'DBScanCluster', 'interdiscipilinary_DB', 'SubUrl', 'Translates'],
      dtype='object')

In [50]:
Merged_df.groupby(['interdiscipilinary_DB','interdiscipilinary_ISO']).count()[['ProjectURL']]

ProjectURL
interdiscipilinary_DB interdiscipilinary_ISO            
False                 False                        48481
                      True                           935
True                  False                         1048
                      True                           569

In [51]:
Merged_df[((Merged_df.interdiscipilinary_DB) & (Merged_df.interdiscipilinary_ISO==False ) )][Feature+['ISOForestCluster']].head(100)

,Translates,Category,Category_1_DB,Category_1_ISO,Category_2_DB,Category_2_ISO,ISOForestCluster
217,The application area ZE Education covers the a...,Natural Sciences,Humanities and Social Sciences,Natural Sciences,Engineering Sciences,NaN,1
449,The aim of this project is to develop efficien...,Natural Sciences,Engineering Sciences,Natural Sciences,Natural Sciences,Engineering Sciences,-1
531,Uncertainty occurs during the development of p...,Natural Sciences,Engineering Sciences,Natural Sciences,Humanities and Social Sciences,NaN,1
532,Statistical modeling is the methodology of con...,Natural Sciences,Humanities and Social Sciences,Natural Sciences,Engineering Sciences,NaN,-1
767,The research program is rooted in geometry. Th...,Natural Sciences,Life Sciences,Natural Sciences,Engineering Sciences,NaN,1
790,"Contents Nowadays, modern technologies in life...",Natural Sciences,Engineering Sciences,Natural Sciences,Humanities and Social Sciences,NaN,-1
918,Vector optimization is a continuously growing ...,Natural Sciences,Engineering Sciences,Natural Sciences,Natural Sciences,Engineering Sciences,-1
952,The ability to invade tissue and form metastas...,Natural Sciences,Life Sciences,Life Sciences,Engineering Sciences,Engineering Sciences,1
988,Biomedical imaging aims at visualizing structu...,Natural Sciences,Engineering Sciences,Natural Sciences,Life Sciences,NaN,-1
1017,In this project we want to analyze finite elem...,Natural Sciences,Engineering Sciences,Engineering Sciences,Natural Sciences,NaN,1


In [52]:
Merged_df[((Merged_df.interdiscipilinary_DB) & (Merged_df.interdiscipilinary_ISO==False ) )]['Translates'].head(100).tolist()

['The application area ZE Education covers the areas of competence training, further education and course development. Our educational competence is based on the experience of teaching students with the subject of mathematics for all types of schools (elementary school, secondary school, high school) and with students of engineering. Wide acclaim has been found in our successful efforts to strengthen the practice orientation of teacher training. In addition, the competence in this field is based on our great successes in the promotion of mathematically gifted and interested students in the "Berlin network of mathematically-scientifically-profiled schools". Continuing education is demonstrated through our innovative approach to teacher education. Its increased national impact is confirmed by the project "Mathematics Anders Machen" funded by the Deutsche Telekom Foundation. Our course development expertise brings together our extensive experience in the development of teaching materials 

In [53]:
Merged_df[((Merged_df.interdiscipilinary_DB) & (Merged_df.interdiscipilinary_ISO ) )][Feature]

,Translates,Category,Category_1_DB,Category_1_ISO,Category_2_DB,Category_2_ISO
107,"In this project, the calcium-regulated force d...",Natural Sciences,Life Sciences,Life Sciences,NaN,Natural Sciences
543,The transregional CRC 73 was initiated to esta...,Natural Sciences,Engineering Sciences,Engineering Sciences,NaN,NaN
559,The aim of this interdisciplinary project is t...,Natural Sciences,Engineering Sciences,Life Sciences,Life Sciences,Natural Sciences
595,"Herein, we describe recent results and future ...",Natural Sciences,Engineering Sciences,Engineering Sciences,Humanities and Social Sciences,Natural Sciences
914,The latest and still ongoing discussion on Die...,Natural Sciences,Engineering Sciences,Humanities and Social Sciences,Natural Sciences,NaN
1060,The sequentially Markov coalescent (SMC) is an...,Natural Sciences,Life Sciences,Life Sciences,NaN,Engineering Sciences
1062,The CRISPR-Cas system (Clustered Regularly Int...,Natural Sciences,Life Sciences,Life Sciences,NaN,NaN
1298,Our project aims to investigate the influence ...,Natural Sciences,Engineering Sciences,Engineering Sciences,NaN,NaN
1847,Historical buildings are components of cultura...,Engineering Sciences,Humanities and Social Sciences,Humanities and Social Sciences,NaN,NaN
2156,Quantitative economics often faces the challen...,Humanities and Social Sciences,Engineering Sciences,Engineering Sciences,Natural Sciences,Humanities and Social Sciences


In [54]:
len(Merged_df[((Merged_df.interdiscipilinary_DB) & (Merged_df.interdiscipilinary_ISO ) )][Feature])

569

In [55]:
Merged_df.columns

Index(['ProjectURL', 'Category', 'Category_1_Values_ISO',
       'Category_2_Values_ISO', 'Category_1_ISO', 'Category_2_ISO',
       'ISOForestCluster', 'interdiscipilinary_ISO', 'Category_1_Values_DB',
       'Category_2_Values_DB', 'Category_1_DB', 'Category_2_DB',
       'DBScanCluster', 'interdiscipilinary_DB', 'SubUrl', 'Translates'],
      dtype='object')

In [56]:
ManualTaggedMerged=manualtag[['Translates', 'Category_1', 'Category_2','interdiscipilinary']].merge(Merged_df,how='left', left_on='Translates', right_on='Translates')

In [57]:
def combine_results(x):
    Actual=x['Category_1']
    cat1=x['Category']
    cat2=np.NaN
    
    if(x['interdiscipilinary_ISO'] and x['interdiscipilinary_DB'] ):
        
    ## Combining both results
               
        if(x['Category_1_DB'] == x['Category_1_ISO']):
            cat1= x['Category_1_DB']
            if(x['Category_2_ISO'] == x['Category_2_DB']):
                cat2= x['Category_2_DB']                
        elif(x['Category_1_DB'] == x['Category_2_ISO']):
            cat1= x['Category_1_DB']
            if(x['Category_2_DB'] == x['Category_1_ISO']):
                cat2= x['Category_2_DB']        
        elif(x['Category_2_DB'] == x['Category_1_ISO']):
            cat1= x['Category_2_DB']        
        else :
            cat1= x['Category']
            
    return pd.Series({'cat1':cat1,'cat2':cat2})
        
ManualTaggedMerged[['Category_1_combined','Category_2_combined']] =ManualTaggedMerged.apply(combine_results, axis=1)

In [58]:

## Function to take a better results for valudation
def rebuilt_ip(x):
    final_pred=x['Category_1_combined']
    final_actual=x['Category_1']
    result=0
    
    if(x['Category_1_combined'] == x['Category_1']):
        final_pred=x['Category_1_combined']
        final_actual=x['Category_1']
        result=1
        
    elif(x['Category_1_combined'] == x['Category_2']):
        final_pred=x['Category_1_combined']
        final_actual=x['Category_2']
        result=1
    
    elif(x['Category_2_combined'] == x['Category_1']):
        final_pred=x['Category_2_combined']
        final_actual=x['Category_1']
        result=.66
        
    elif(x['Category_2_combined'] == x['Category_2']):
        final_pred=x['Category_2_combined']
        final_actual=x['Category_2']
        result=.66

    return pd.Series({'pred':final_pred,'actual':final_actual,'result':result})


ManualTaggedMerged[['Predicted','Actual','Result']]=ManualTaggedMerged.apply(rebuilt_ip, axis=1)

In [59]:
# Print the confusion matrix
print(metrics.confusion_matrix(ManualTaggedMerged.Actual, ManualTaggedMerged.Predicted))

# Print the precision and recall, among other metrics
print(metrics.classification_report(ManualTaggedMerged.Actual, ManualTaggedMerged.Predicted, digits=3))

[[12  0  0  1]
 [ 1 11  0  0]
 [ 1  3 11  2]
 [ 0  0  0  8]]
                                precision    recall  f1-score   support

          Engineering Sciences      0.857     0.923     0.889        13
Humanities and Social Sciences      0.786     0.917     0.846        12
                 Life Sciences      1.000     0.647     0.786        17
              Natural Sciences      0.727     1.000     0.842         8

                      accuracy                          0.840        50
                     macro avg      0.843     0.872     0.841        50
                  weighted avg      0.868     0.840     0.836        50



## Used to generate the manual tagging records 

ID= ((Merged_df.interdiscipilinary_DB) & (Merged_df.interdiscipilinary_ISO ) )
Not_ID= ((Merged_df.interdiscipilinary_DB==False) & (Merged_df.interdiscipilinary_ISO==False) )
False_positivies= (Merged_df.DBScanCluster!=-1) & (Merged_df.ISOForestCluster!=-1) & (Merged_df.Category!=Merged_df.Category_1_DB) & (Merged_df.Category!=Merged_df.Category_1_ISO)

Samples=pd.DataFrame()

for cat in Merged_df.Category.unique():
    cat_filter=(Merged_df.Category==cat)
    
    Samples=pd.concat([Samples,Merged_df[(cat_filter) & (ID)].sample(n=24)], ignore_index=True)
    Samples=pd.concat([Samples,Merged_df[(cat_filter) & (Not_ID)].sample(n=24)], ignore_index=True)
    Samples=pd.concat([Samples,Merged_df[(cat_filter) & (False_positivies)].sample(n=8)], ignore_index=True)

Final_sample=Samples.sample(50)
Final_sample.Category.value_counts()
Final_sample.interdiscipilinary_ISO.value_counts()
#Final_sample[[ 'Category','Translates']].to_csv(Path+'SamplesManualTagger.csv', index=False)